# Environment

In [37]:
import numpy as np, pandas as pd, tensorflow as tf

from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession

In [38]:
BATCH_SIZE = 4
BUFFER_SIZE = 100

# Function Definitions

In [29]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop("target")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

# BigQuery Connector

Documentation: https://github.com/tensorflow/io/tree/master/tensorflow_io/bigquery

In [20]:
GCP_PROJECT_ID = "axa-ch-machine-learning-dev"
DATASET_GCP_PROJECT_ID = "axa-ch-machine-learning-dev"
DATASET_ID = "nlp_text_classification"
TABLE_ID = "stackoverflow_posts"

In [54]:
# ops.enable_eager_execution()
client = BigQueryClient()

read_session = client.read_session(
    "projects/" + GCP_PROJECT_ID,
    DATASET_GCP_PROJECT_ID,
    TABLE_ID,
    DATASET_ID,
    ["body", "tags"],
    [dtypes.string, dtypes.string],
    row_restriction="",
)
dataset = (
    read_session.parallel_read_rows()
    .shuffle(buffer_size=BUFFER_SIZE)
    .batch(batch_size=BATCH_SIZE)
)

# Refactor the Input

In [56]:
COLUMNS = ['body', 'tags']

# Keras

In [47]:
EPOCHS = 10

## Creating a Feature Layer

In [ ]:
feature_columns = []

In [48]:
for header in ["body"]:
    feature_columns.append(tf.feature_column.numeric_column(header))

feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

## Model

In [50]:
model = tf.keras.Sequential(
    [
        feature_layer,
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

In [51]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [52]:
model.fit(gcp_dataset, epochs=EPOCHS, steps_per_epoch=100)

Train for 100 steps
Epoch 1/10
  1/100 [..............................] - ETA: 4s

IndexError: list index out of range

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)